In [6]:
filepath='/Users/stephaniexia/Documents/lab/data/2018-6-24_rear_dist_onroad_AnnArbor8_Annotations/2018-6-24_rear_dist_onroad_AnnArbor8-00031.xml'

In [108]:
import xml.etree.ElementTree as ET
import numpy as np
import tensorflow as tf
import os

def get_cls_and_boxes(filepath,cls_dic,img_width,img_hieght):
    #get class ids and boxes from xml annotations:
    cls_labels=[]
    boxes=[]
    #box:[y1,x1,y2,x2]
    annos=ET.ElementTree(file=filepath)
    h, w = float(img_width),(img_hieght)
    image_scale = np.array([h, w, h, w])
    for elem in annos.iterfind('object'):
        cls_labels.append(elem.findtext('name'))
        for nodes in elem.iterfind('bndbox'):
            _box=np.array([float(nodes.findtext('ymax')),float(nodes.findtext('xmax')),
                           float(nodes.findtext('ymin')),float(nodes.findtext('xmin'))])
            # Normalize coordinates
            _box=(_box/image_scale).tolist()
            boxes.append(_box)
    #change class labels to numeric ids
    cls_ids=[cls_dic[x] if x in cls_dic else x for x in cls_labels]
    return cls_ids,boxes

def padding_gt(cls_ids,boxes,num_boxes,cls_id_lst):
    #keep interested ids and boxes 
    _gt_cls_ids=[cls_ids[i] for i in range(len(cls_ids)) if cls_ids[i] in cls_id_lst]
    _gt_boxes=[boxes[i] for i in range(len(cls_ids)) if cls_ids[i] in cls_id_lst]
    print(_gt_cls_ids,_gt_boxes)
    #padding zeros 
    if len(_gt_cls_ids) < num_boxes:
        gt_cls_ids=np.zeros((num_boxes))
        gt_boxes=np.zeros((num_boxes,4))
        for i in range(len(_gt_cls_ids)):
            gt_cls_ids[i]=_gt_cls_ids[i]
            gt_boxes[i]=_gt_boxes[i]
    elif len(_gt_cls_ids) == num_boxes:
        gt_cls_ids=_gt_cls_ids
        gt_boxes=_gt_boxes
    #or randomly choose num_boxes:
    elif len(_gt_cls_ids) > num_boxes:
        gt_cls_ids=np.zeros((num_boxes))
        gt_boxes=np.zeros((num_boxes,4))
        list=[i for i in range(len(_gt_cls_ids))]
        choose_ix=random.choice(list,5).tolist()
        for i in range(len(choose_ix)):
            ix=choose_ix[i]
            gt_cls_ids[i]=_gt_cls_ids[ix]
            gt_boxes[i]=_gt_boxes[ix]
    return gt_cls_ids,gt_boxes

def get_gts_for_dir(anno_dir,cls_dic,img_width,img_hieght,num_boxes,cls_id_lst):
    #get gts for a dir
    #by calling functions: get_cls_and_boxes & padding_gt
    xml_list =os.listdir(anno_dir)
    Y_gt_cls_ids=np.zeros((len(xml_list),num_boxes))
    Y_gt_boxes=np.zeros((len(xml_list),num_boxes,4))
    for i in range(len(xml_list)):
        anno_path = os.path.join(anno_dir, xml_list[i])
        cls_ids,boxes=get_cls_and_boxes(anno_path,cls_dic,img_width,img_hieght)
        gt_cls_ids,gt_boxes=padding_gt(cls_ids,boxes,num_boxes,cls_id_lst)
        Y_gt_cls_ids[i]=gt_cls_ids
        Y_gt_boxes[i]=gt_boxes
    return Y_gt_cls_ids, Y_gt_boxes

num_boxes=5
cls_dic={'car':1,'truck':2,'pedestrian':3,'stop_sign':4,'traffic_light':5,'motorbike':6,'bicycle':7}   
cls_id_lst=[3]
img_width,img_hieght=1280,1024
anno_dir='/Users/stephaniexia/Documents/lab/data/2018-6-24_rear_dist_onroad_AnnArbor8_Annotations'

Y_gt_cls_ids, Y_gt_boxes=get_gts_for_dir(anno_dir,cls_dic,img_width,img_hieght,num_boxes,cls_id_lst)
print(Y_gt_cls_ids.shape,'\n',Y_gt_boxes.shape)
        #print(nodes.tag,nodes.text)

[] []
[] []
[] []
[] []
[] []
[3] [[0.27890625, 0.216796875, 0.32578125, 0.2421875]]
[3] [[0.246875, 0.3759765625, 0.27890625, 0.392578125]]
[3] [[0.24453125, 0.8671875, 0.28046875, 0.888671875]]
[3] [[0.25234375, 0.85546875, 0.2875, 0.873046875]]
[3, 3] [[0.27421875, 1.1552734375, 0.4265625, 1.20703125], [0.24609375, 1.1044921875, 0.42421875, 1.169921875]]
[] []
[] []
[] []
[3, 3, 3] [[0.240625, 0.7822265625, 0.275, 0.798828125], [0.2734375, 0.1611328125, 0.32109375, 0.189453125], [0.23046875, 0.703125, 0.2609375, 0.716796875]]
[3, 3, 3, 3] [[0.21484375, 0.9404296875, 0.37265625, 0.99609375], [0.24140625, 0.759765625, 0.2734375, 0.771484375], [0.24140625, 0.76953125, 0.27890625, 0.78515625], [0.2390625, 0.5390625, 0.26484375, 0.552734375]]
[3, 3, 3, 3, 3] [[0.29140625, 0.1552734375, 0.35234375, 0.189453125], [0.34296875, 1.115234375, 0.42265625, 1.158203125], [0.30390625, 0.142578125, 0.35390625, 0.166015625], [0.240625, 0.458984375, 0.26796875, 0.474609375], [0.23515625, 0.6435546875

[] []
[3, 3] [[0.24296875, 0.8466796875, 0.28125, 0.8681640625], [0.23984375, 0.4033203125, 0.27109375, 0.4189453125]]
[3, 3] [[0.24296875, 0.455078125, 0.26875, 0.4697265625], [0.2421875, 0.4384765625, 0.2703125, 0.455078125]]
[] []
[] []
[] []
[3, 3, 3] [[0.2578125, 0.3232421875, 0.28984375, 0.337890625], [0.25546875, 0.333984375, 0.2890625, 0.349609375], [0.24375, 0.4423828125, 0.26875, 0.4560546875]]
[3] [[0.2421875, 0.87109375, 0.290625, 0.8896484375]]
[] []
[] []
(281, 5) 
 (281, 5, 4)


In [105]:
list=[2,4,6,8]
a=float(list)
a

TypeError: float() argument must be a string or a number, not 'list'

In [86]:
_gt_cls_ids=random.random((6))
_gt_boxes=random.random((6,4))
print(_gt_cls_ids,_gt_boxes)


gt_cls_ids=np.zeros((num_boxes))
gt_boxes=np.zeros((num_boxes,4))
list=[i for i in range(len(_gt_cls_ids))]
choose_ix=random.choice(list,5).tolist()
for i in range(len(choose_ix)):
    ix=choose_ix[i]
    gt_cls_ids[i]=_gt_cls_ids[ix]
    gt_boxes[i]=_gt_boxes[ix]
gt_cls_ids
gt_boxes

    

[0.87467718 0.86087086 0.338708   0.06228214 0.02067591 0.93248785] [[0.61730143 0.11037916 0.80581828 0.8657838 ]
 [0.57080707 0.14172424 0.06134486 0.74580705]
 [0.85090092 0.30854077 0.05940003 0.44820318]
 [0.79524092 0.11926126 0.97766126 0.40683307]
 [0.16477129 0.25419968 0.5692627  0.1461658 ]
 [0.00311674 0.5525667  0.22297608 0.88208368]]


array([[0.16477129, 0.25419968, 0.5692627 , 0.1461658 ],
       [0.00311674, 0.5525667 , 0.22297608, 0.88208368],
       [0.00311674, 0.5525667 , 0.22297608, 0.88208368],
       [0.79524092, 0.11926126, 0.97766126, 0.40683307],
       [0.85090092, 0.30854077, 0.05940003, 0.44820318]])


I. car
II. truck
III. pedestrian
IV. stop_sign
V. traffic_light
VI. motorbike
VII. bicycle

<annotation>
	<folder>2018-6-24_rear_dist_onroad_AnnArbor8</folder>
	<filename>2018-6-24_rear_dist_onroad_AnnArbor8-00031.jpg</filename>
	<path>/Users/stephaniexia/Downloads/2018-6-24_rear_dist_onroad_AnnArbor8/2018-6-24_rear_dist_onroad_AnnArbor8-00031.jpg</path>
	<source>
		<database>Unknown</database>
	</source>
	<size>
		<width>1280</width>
		<height>1024</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>car</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>143</xmin>
			<ymin>369</ymin>
			<xmax>258</xmax>
			<ymax>438</ymax>
		</bndbox>
	</object>
</annotation>
